This notebook demonstrates a prototype of a TRX file that leverages the parquet format.

In [82]:
from trx_parquet import trxparquet

The primary class is `TrxParquet`. It contains two key attributes, `header` and `data`. The attribute `header` aims to store the minimal amount of information necessary for processing and conversion to other formats (e.g., to a `StatefulTractogram`) . The `data` attribute contains most of the tractography information.

To get started, see `init_example_trxparquet`, which initializes in-memory representations of trxparquet files with given characteristics and random data. For example, we can initialize two streamlines, each having 3 points.

In [83]:
trxparquet.init_example_trxparquet(2, 3)

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 4)
┌──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┐
│ protected_streamline ┆ protected_position_0 ┆ protected_position_1 ┆ protected_position_2 │
│ ---                  ┆ ---                  ┆ ---                  ┆ ---                  │
│ i64                  ┆ f64                  ┆ f64                  ┆ f64                  │
╞══════════════════════╪══════════════════════╪══════════════════════╪══════════════════════╡
│ 0                    ┆ 0.485554             ┆ 0.472543             ┆ 0.424029             │
│ 0                    ┆ 0.116149             ┆ 0.947387             ┆ 0.245581             │
│ 0                    ┆ 0.558482             ┆ 0.912673             ┆ 0.980443             │
│ 1   

In the trxparquet itself (on disk and in memory), the `header` is stored via frame-level metadata (e.g., of the kind readable by https://arrow.apache.org/docs/python/generated/pyarrow.parquet.read_metadata.html). 

The `data` attribute always contains at least four columns, each of which have the prefix "protected_". These columns represent 
- An index for streamline
- 3 columns representing the coordinates of each point/vertex within each streamline. 

That is, rows in the data correspond to points or vertices.

Data that is associated with each streamline is stored in columns that have the prefix "dps_". The function `init_example_trxparquet` can be used to create a this kind of column. The column label includes a random string.

In [84]:
trxparquet.init_example_trxparquet(2, 3, 1)

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 5)
┌─────────────────────┬─────────────────────┬────────────────────┬────────────────────┬────────────┐
│ protected_streamlin ┆ protected_position_ ┆ protected_position ┆ protected_position ┆ dps_pddfst │
│ e                   ┆ 0                   ┆ _1                 ┆ _2                 ┆ ---        │
│ ---                 ┆ ---                 ┆ ---                ┆ ---                ┆ f64        │
│ i64                 ┆ f64                 ┆ f64                ┆ f64                ┆            │
╞═════════════════════╪═════════════════════╪════════════════════╪════════════════════╪════════════╡
│ 0                   ┆ 0.51418             ┆ 0.605899           ┆ 0.609424           ┆ 0.479546   │
│ 0                   ┆ 0.748811            ┆ 0.770

Analogously, data that is associated with individual points will have the prefix "dpv_".

In [85]:
trx = trxparquet.init_example_trxparquet(2, 3, 1, 1)
trx

TrxParquet(header=TrxHeader(DIMENSIONS=array([20, 20, 20], dtype=uint16), VOXEL_TO_RASMM=array([[20.,  0.,  0.,  0.],
       [ 0., 20.,  0.,  0.],
       [ 0.,  0., 20.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)), data=shape: (6, 6)
┌──────────────────┬─────────────────┬─────────────────┬─────────────────┬────────────┬────────────┐
│ protected_stream ┆ protected_posit ┆ protected_posit ┆ protected_posit ┆ dps_uuhdni ┆ dpv_uvbung │
│ line             ┆ ion_0           ┆ ion_1           ┆ ion_2           ┆ ---        ┆ ---        │
│ ---              ┆ ---             ┆ ---             ┆ ---             ┆ f64        ┆ f64        │
│ i64              ┆ f64             ┆ f64             ┆ f64             ┆            ┆            │
╞══════════════════╪═════════════════╪═════════════════╪═════════════════╪════════════╪════════════╡
│ 0                ┆ 0.280597        ┆ 0.310004        ┆ 0.320914        ┆ 0.54114    ┆ 0.7737     │
│ 0                ┆ 0.970233        ┆ 0.542194    

The class definition currently includes examples of how to convert `data` into other formats. For example, we can check the property `data_per_streamline`.

In [86]:
trx.data_per_streamline

{'uuhdni': array([0.54113962, 0.89981118])}

These objects can also be converted into `StatefulTractogram`s using the `to_stf()` method.

In [87]:
stf = trxparquet.init_example_trxparquet(2, 3, 1, 1).to_stf()
stf.streamlines

ArraySequence([array([[0.28763362, 0.24509462, 0.78050736],
       [0.70758868, 0.96662439, 0.63242437],
       [0.51510291, 0.77292236, 0.88291752]]), array([[0.67614554, 0.59743025, 0.12807905],
       [0.04290932, 0.86055621, 0.05215408],
       [0.0484068 , 0.55349278, 0.49419084]])])

The assumption is that, by relying on the parquet format, we get get to leverage all of the work that has gone into making this an efficient medium for analysis. For example, let's create a file that has 100000 streamlines, each with 100 points, checking the size of the file and how long it takes to load as a memory map.

In [88]:
import tempfile
import time
from pathlib import Path


def human_size(bytes, units=[" bytes", "KB", "MB", "GB", "TB", "PB", "EB"]):
    """Returns a human readable string representation of bytes"""
    return (
        str(bytes) + units[0]
        if bytes < 1024
        else human_size(bytes >> 10, units[1:])
    )


with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    size = human_size(f.stat().st_size)
    start = time.time()
    trx2 = trx.from_file(f, loadtype="memory_map")
    end = time.time()

print(f"size: {size}")
print(f"reading time: {end - start}")

trx2
del trx, trx2

size: 230MB
reading time: 0.08661866188049316


There are different ways of loading parquet files, each optimized for different purposes. The previous cell loaded files as a memory map. If only some streamlines need to be processed (or only some columns), then Lazy loading has many advantages. See: https://pola-rs.github.io/polars/user-guide/concepts/lazy-vs-eager/ . The reading time for lazy loading is minimal, but we can still extract useful information from the result.

In [89]:
with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    start = time.time()
    n_streamlines_in_file = trx.from_file(f, loadtype="lazy").n_streamlines
    end = time.time()

print(f"reading time: {end - start}")
print(f"{n_streamlines_in_file=}")
del trx

reading time: 0.05006289482116699
n_streamlines_in_file=100000


Even when files are loaded into memory, the process is remains speedy.

In [90]:
with tempfile.NamedTemporaryFile(suffix=".parquet") as _f:
    f = Path(_f.name)
    trx = trxparquet.init_example_trxparquet(100000, 100)
    trx.to_file(f)
    start = time.time()
    trx2 = trx.from_file(f, loadtype="memory")
    end = time.time()

object_size = trx2.data.estimated_size("mb")
print(f"{object_size=} MB")
print(f"reading time: {end - start}")
del trx, trx2

object_size=305.17578125 MB
reading time: 0.09228920936584473


For a few additional examples, please see the tests.

Note that, at the time of writing, no group-level information has been incorporated into the prototype.